In [1]:
import json
import os
import numpy as np

def configInfo(file):
    with open(file, 'r', encoding='utf-8') as f:
        config = json.load(f)
    return config

def imgList(data, config='config.json'):

    thumnail_path = configInfo(config)["path"]["thumnail"]
    img_list, img_list2 = [], []

    for img in data.imgs:
        img_list.append(img[0])

    for img in os.listdir(thumnail_path):
        img_list2.append(os.path.join(thumnail_path,img))
    img_list2.sort()

    return img_list, img_list2

def avgList(result, label_arr):

    avg_list = []
    scatter_x = result[:, 0]
    scatter_y = result[:, 1]
    group = np.array(label_arr)

    for g in np.unique(group):
        i = np.where(group == g)
        x_avg = np.mean(scatter_x[i])
        y_avg = np.mean(scatter_y[i])
        avg_list.append((x_avg, y_avg))

    return 

In [5]:
import os
import numpy as np
import glob
from PIL import Image


def getCoord(image, height, width, std):

    mid = width // 2
    coor = []
    final = []
    cnt = 0

    for i in range(height):
        if image[i, mid, 0] == 255 and image[i, mid, 1] == 255 and image[i, mid, 2] == 255:
            continue
        if coor:
            if abs(coor[-1] - i) > std:
                final.append((coor[0], coor[-1]))
                cnt += 1
                coor = []
            else:
                coor.append(i)
        else:
            coor.append(i)

        if i == height-1:
            start, end = coor[0], coor[-1]
            final.append((start, end))
            cnt += 1

    return cnt, final


def Croptoon(path, save_dir, std=150):
    hap = 0
    i = 0

    for file in glob.glob(path + '/*'):
        try:
            image = np.asarray(Image.open(file))
            cnt, final = getCoord(image, image.shape[0], image.shape[1], std)
            hap += cnt
            for (start, end) in final:
                cropped = image[start:end, :]
                if cropped.shape[0] < 250:
                    hap -= 1
                    continue
                cropped = Image.fromarray(cropped)
                cropped.save(save_dir + '/' + str(i) + ".jpg")
                i += 1
            print(f'{file} cropped => {cnt} images')
        except:
            continue

    print(f'Total {hap} images cropped')

if __name__ == '__main__':

    webtoon_path = "D:webtoons/"
    cropped_path = "D:CropWebtoons/"

    for dir in os.listdir(webtoon_path):
        os.makedirs(os.path.join(os.getcwd(), cropped_path, dir))

    for toon in os.listdir(webtoon_path):
        Croptoon(webtoon_path + toon, cropped_path + toon)

PermissionError: [Errno 13] Permission denied: 'D:webtoons/108명의 그녀들\\1'